In [50]:
import numpy as np
import pandas as pd
import torch
pd.options.display.max_columns = None
pd.options.display.max_rows = None

import matplotlib.pyplot as plt
%matplotlib inline

import datetime as dt
import pickle
import os
import time

import seaborn as sns
sns.set()
from matplotlib import style
# style.use('ggplot')
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
df = pd.read_csv('data_scaled.csv')
# df = df.drop(columns=["Return"])
df['Date'] = pd.to_datetime(df['Date'])

In [40]:
# def cat_cont_split(df, maxcard=55, omit_vars=['Date', 'Adj Close', 'Return', 'target_price']):
def cat_cont_split(df, maxcard=55, omit_vars=['Date', 'target_price']):
    """Helper function that returns column names of categorical & continuous features from df."""
    cat_feats, cont_feats = [], []
    for col in df:
        if col in omit_vars: 
            continue
        if (df[col].dtype==int or df[col].dtype==float) and df[col].unique().shape[0] > maxcard:
            cont_feats.append(col)
        else:
            cat_feats.append(col)
    return cat_feats, cont_feats
    
cat_vars, cont_vars = cat_cont_split(df)
print(len(cat_vars), 'Categorical Features:')
print(cat_vars)
print(len(cont_vars), 'Continuous Features:')
print(cont_vars)

class Categorifier:
    ''' Transform categorical features into category types '''
    def apply_train(self, df, cat_vars):
        self.cat_vars = cat_vars
        self.categories = {}
        for v in self.cat_vars:
            df.loc[:, v] = df.loc[:, v].astype('category').cat.as_ordered()
            self.categories[v] = df[v].cat.categories
            
    def apply_test(self, df_test):
        for v in self.cat_vars:
            df_test.loc[:, v] = pd.Categorical(df[v], categories=self.categories[v], ordered=True)

cat = Categorifier()
cat.apply_train(df, cat_vars)
df.info()

19 Categorical Features:
['volatility_bbhi', 'volatility_bbli', 'trend_psar_up_indicator', 'trend_psar_down_indicator', 'ticker', 'exchange', 'sector', 'industry', 'Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
13 Continuous Features:
['Adj Close', 'volume_obv', 'volume_fi', 'volatility_bbm', 'volatility_bbw', 'trend_macd', 'trend_macd_signal', 'trend_macd_diff', 'momentum_ao', 'momentum_roc', 'Return', 'Dayofyear', 'Elapsed']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1197235 entries, 0 to 1197234
Data columns (total 34 columns):
 #   Column                     Non-Null Count    Dtype         
---  ------                     --------------    -----         
 0   Date                       1197235 non-null  datetime64[ns]
 1   Adj Close                  1197235 non-null  float64       
 2   volume_obv                 1197235 non-null  float64       
 3   volume_fi               

In [41]:
dep_var = 'target_price'
cont_vars = cont_vars
cat_sz = [(nc, len(df[nc].cat.categories)+1) for nc in cat_vars]
# emb_szs = [(nc, min(50, (nc+1)//2)) for _, nc in cat_sz]
emb_szs = [(nc, min(51, round(1.6*nc**0.56))) for _, nc in cat_sz]

In [6]:
train_arrs, test_arrs = [], []

arr_names = ["Xhist", "Xcat", "Xcont", "y", "dateticker"]

for arr_name in arr_names:
    fname = 'data/temporal_data/' + arr_name + "Train.npy"
    train_arrs.append(np.load(fname, allow_pickle=True))
    fname = 'data/temporal_data/' + arr_name + "Test.npy"
    test_arrs.append(np.load(fname, allow_pickle=True))
    
XseqTrain, XcatTrain, XcontTrain, yTrain, datetickerTrain = train_arrs
XseqTest, XcatTest, XcontTest, yTest, datetickerTest = test_arrs

print(XseqTrain.shape, XseqTest.shape, yTrain.shape, yTest.shape)

(1106055, 128) (28615, 128) (1106055, 1) (28615, 1)


# Stacking TCN

In [17]:
import stacking_tcn
from torch.utils.data import DataLoader

bs = 64
timesteps = 128

train_ds = stacking_tcn.HybridDataset(XcatTrain, XcontTrain, XseqTrain, yTrain, timesteps)
test_ds = stacking_tcn.HybridDataset(XcatTest, XcontTest, XseqTest, yTest, timesteps)

train_dl = DataLoader(train_ds, bs, shuffle=True)
val_dl = DataLoader(test_ds, 1024, shuffle=True)

In [42]:
cmodel = stacking_tcn.StackingTCN(emb_szs, len(cont_vars), hidden_sizes=[1024, 512], out_size=1,
                  emb_dropout = 0.1, dropout_prob = [0.001, .01], use_bn=True,
                  cnn_input_size=1, num_kernels=32, kernel_size=2,
                  cnn_num_blocks=7, cnn_output_size=1, cnn_dropout=0.2)
print(cmodel)

StackingTCN(
  (tcn): TemporalConvNet(
    (network): Sequential(
      (0): TemporalBlock(
        (conv1): Conv1d(1, 32, kernel_size=(2,), stride=(1,), padding=(1,))
        (chomp1): Chomp1d()
        (relu1): ReLU()
        (dropout1): Dropout(p=0.2, inplace=False)
        (conv2): Conv1d(32, 32, kernel_size=(2,), stride=(1,), padding=(1,))
        (chomp2): Chomp1d()
        (relu2): ReLU()
        (dropout2): Dropout(p=0.2, inplace=False)
        (net): Sequential(
          (0): Conv1d(1, 32, kernel_size=(2,), stride=(1,), padding=(1,))
          (1): Chomp1d()
          (2): ReLU()
          (3): Dropout(p=0.2, inplace=False)
          (4): Conv1d(32, 32, kernel_size=(2,), stride=(1,), padding=(1,))
          (5): Chomp1d()
          (6): ReLU()
          (7): Dropout(p=0.2, inplace=False)
        )
        (downsample): Conv1d(1, 32, kernel_size=(1,), stride=(1,))
        (relu): ReLU()
      )
      (1): TemporalBlock(
        (conv1): Conv1d(32, 32, kernel_size=(2,), stride=

In [20]:
stacking_tcn.train_model(cmodel, val_dl, val_dl, n_epochs=1, lr=1e-2)

100%|██████████| 28/28 [00:04<00:00,  6.09it/s]


Epoch 1: training loss=0.15789
Epoch 1: val loss=0.05063


In [21]:
train_preds, train_targets = stack_tcn.predict(cmodel, train_ds, 512)
test_preds, test_targets = stack_tcn.predict(cmodel, test_ds, 512)

100%|██████████| 56/56 [00:00<00:00, 63.11it/s]


# Stacking LSTM

In [24]:
import stacking_lstm
from torch.utils.data import DataLoader

bs = 64
timesteps = 50

train_ds = stacking_lstm.HybridDataset(XcatTrain, XcontTrain, XseqTrain, yTrain, timesteps)
test_ds = stacking_lstm.HybridDataset(XcatTest, XcontTest, XseqTest, yTest, timesteps)

train_dl = DataLoader(train_ds, bs, shuffle=True)
val_dl = DataLoader(test_ds, 1024, shuffle=True)

In [98]:
rmodel = stacking_lstm.StackingLSTM(emb_szs, len(cont_vars), hidden_sizes=[1024, 512], 
                emb_dropout = 0.1, dropout_prob = [0.001, .01], use_bn=True,
                rnn_input_size=1, rnn_hidden_size=30, rnn_num_layers=3,
                rnn_output_size=1, rnn_dropout=0.)
print(rmodel)

StackingLSTM(
  (rnn): LSTM(1, 30, num_layers=3, batch_first=True)
  (fc): Linear(in_features=30, out_features=1, bias=True)
  (embeds): ModuleList(
    (0): Embedding(3, 3)
    (1): Embedding(3, 3)
    (2): Embedding(3, 3)
    (3): Embedding(3, 3)
    (4): Embedding(486, 51)
    (5): Embedding(3, 3)
    (6): Embedding(13, 7)
    (7): Embedding(105, 22)
    (8): Embedding(12, 6)
    (9): Embedding(13, 7)
    (10): Embedding(54, 15)
    (11): Embedding(32, 11)
    (12): Embedding(6, 4)
    (13): Embedding(3, 3)
    (14): Embedding(3, 3)
    (15): Embedding(3, 3)
    (16): Embedding(3, 3)
    (17): Embedding(3, 3)
    (18): Embedding(2, 2)
  )
  (emb_dropout): Dropout(p=0.1, inplace=False)
  (bn_cont): BatchNorm1d(13, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layers): Sequential(
    (0): Linear(in_features=168, out_features=1024, bias=True)
    (1): ReLU(inplace=True)
    (2): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    

In [39]:
stacking_lstm.train_model(rmodel, val_dl, val_dl, n_epochs=10, lr=1e-2)

100%|██████████| 28/28 [00:02<00:00, 13.16it/s]


Epoch 1: training loss=1.63245


  7%|▋         | 2/28 [00:00<00:02, 11.25it/s]

Epoch 1: val loss=1.18611


100%|██████████| 28/28 [00:01<00:00, 14.49it/s]


...

Epoch 9: training loss=0.05636


  7%|▋         | 2/28 [00:00<00:02, 12.61it/s]

Epoch 9: val loss=0.05479


100%|██████████| 28/28 [00:02<00:00, 13.35it/s]


Epoch 10: training loss=0.05409
Epoch 10: val loss=0.05355


In [36]:
train_preds, train_targets = stacking_lstm.predict(rmodel, train_ds, 512)
test_preds, test_targets = stacking_lstm.predict(rmodel, test_ds, 512)

100%|██████████| 56/56 [00:00<00:00, 58.38it/s]


In [59]:
rmodel

StackingLSTM(
  (rnn): LSTM(1, 30, num_layers=3, batch_first=True)
  (fc_rnn): Linear(in_features=30, out_features=1, bias=True)
  (embeds): ModuleList(
    (0): Embedding(3, 3)
    (1): Embedding(3, 3)
    (2): Embedding(3, 3)
    (3): Embedding(3, 3)
    (4): Embedding(486, 51)
    (5): Embedding(3, 3)
    (6): Embedding(13, 7)
    (7): Embedding(105, 22)
    (8): Embedding(12, 6)
    (9): Embedding(13, 7)
    (10): Embedding(54, 15)
    (11): Embedding(32, 11)
    (12): Embedding(6, 4)
    (13): Embedding(3, 3)
    (14): Embedding(3, 3)
    (15): Embedding(3, 3)
    (16): Embedding(3, 3)
    (17): Embedding(3, 3)
    (18): Embedding(2, 2)
  )
  (emb_dropout): Dropout(p=0.1, inplace=False)
  (bn_cont): BatchNorm1d(13, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc_layers): Sequential(
    (0): Linear(in_features=168, out_features=1024, bias=True)
    (1): ReLU(inplace=True)
    (2): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=Tru

In [101]:
path = './models/ts_lstm.mod'
saved_lstm = torch.load(path)
saved_lstm.keys()

odict_keys(['rnn.weight_ih_l0', 'rnn.weight_hh_l0', 'rnn.bias_ih_l0', 'rnn.bias_hh_l0', 'rnn.weight_ih_l1', 'rnn.weight_hh_l1', 'rnn.bias_ih_l1', 'rnn.bias_hh_l1', 'rnn.weight_ih_l2', 'rnn.weight_hh_l2', 'rnn.bias_ih_l2', 'rnn.bias_hh_l2', 'fc.weight', 'fc.bias'])

In [103]:
def transfer_statedict(model, saved_modules):
    own_state = model.state_dict()
    for state_dict in saved_modules:
        for name, param in state_dict.items():
            if name not in own_state:
                print(name)
                continue
            if isinstance(param, torch.nn.Parameter):
                param = param.data
            own_state[name].copy_(param)
            
transfer_statedict(rmodel, [saved_stock2vec, saved_lstm])

In [106]:
rmodel

StackingLSTM(
  (rnn): LSTM(1, 30, num_layers=3, batch_first=True)
  (fc): Linear(in_features=30, out_features=1, bias=True)
  (embeds): ModuleList(
    (0): Embedding(3, 3)
    (1): Embedding(3, 3)
    (2): Embedding(3, 3)
    (3): Embedding(3, 3)
    (4): Embedding(486, 51)
    (5): Embedding(3, 3)
    (6): Embedding(13, 7)
    (7): Embedding(105, 22)
    (8): Embedding(12, 6)
    (9): Embedding(13, 7)
    (10): Embedding(54, 15)
    (11): Embedding(32, 11)
    (12): Embedding(6, 4)
    (13): Embedding(3, 3)
    (14): Embedding(3, 3)
    (15): Embedding(3, 3)
    (16): Embedding(3, 3)
    (17): Embedding(3, 3)
    (18): Embedding(2, 2)
  )
  (emb_dropout): Dropout(p=0.1, inplace=False)
  (bn_cont): BatchNorm1d(13, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layers): Sequential(
    (0): Linear(in_features=168, out_features=1024, bias=True)
    (1): ReLU(inplace=True)
    (2): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    

In [107]:
rmodel.ensemb.parameters()

<generator object Module.parameters at 0x7f47abaec3b8>

In [81]:
# own_state = rmodel.state_dict()
# for name, param in saved.items():
#     if name not in own_state:
#         print(name)
#         continue
#     if isinstance(param, torch.nn.Parameter):
#         param = param.data
#     own_state[name].copy_(param)

In [83]:
# rmodel.rnn.state_dict().keys()

odict_keys(['weight_ih_l0', 'weight_hh_l0', 'bias_ih_l0', 'bias_hh_l0', 'weight_ih_l1', 'weight_hh_l1', 'bias_ih_l1', 'bias_hh_l1', 'weight_ih_l2', 'weight_hh_l2', 'bias_ih_l2', 'bias_hh_l2'])

In [86]:
# rmodel.state_dict()['embeds.6.weight']
# saved_rnn.keys()

odict_keys(['rnn.weight_ih_l0', 'rnn.weight_hh_l0', 'rnn.bias_ih_l0', 'rnn.bias_hh_l0', 'rnn.weight_ih_l1', 'rnn.weight_hh_l1', 'rnn.bias_ih_l1', 'rnn.bias_hh_l1', 'rnn.weight_ih_l2', 'rnn.weight_hh_l2', 'rnn.bias_ih_l2', 'rnn.bias_hh_l2', 'fc.weight', 'fc.bias'])

In [91]:
# own_state = rmodel.state_dict()
# for name, param in saved_rnn.items():
#     if name not in own_state:
#         print(name)
#         continue
#     if isinstance(param, torch.nn.Parameter):
#         param = param.data
#     own_state[name].copy_(param)

In [104]:
rmodel.rnn.state_dict()

OrderedDict([('weight_ih_l0',
              tensor([[ -1.6964],
                      [ -1.1273],
                      [ -1.4886],
                      [ -1.0371],
                      [  1.3325],
                      [ -4.8691],
                      [ -1.4506],
                      [ -3.9815],
                      [ -1.9023],
                      [ -3.6095],
                      [ -1.8696],
                      [ -3.1493],
                      [  2.1087],
                      [ -1.0278],
                      [ -2.1997],
                      [ -9.7966],
                      [ -3.5947],
                      [ -2.5994],
                      [ -1.5737],
                      [  0.2767],
                      [ -0.8013],
                      [ -3.3813],
                      [ -6.0551],
                      [ -1.1033],
                      [ -4.2144],
                      [ -2.0643],
                      [ -1.2207],
                      [ -5.2666],
                  

In [105]:
saved_lstm

OrderedDict([('rnn.weight_ih_l0',
              tensor([[ -1.6964],
                      [ -1.1273],
                      [ -1.4886],
                      [ -1.0371],
                      [  1.3325],
                      [ -4.8691],
                      [ -1.4506],
                      [ -3.9815],
                      [ -1.9023],
                      [ -3.6095],
                      [ -1.8696],
                      [ -3.1493],
                      [  2.1087],
                      [ -1.0278],
                      [ -2.1997],
                      [ -9.7966],
                      [ -3.5947],
                      [ -2.5994],
                      [ -1.5737],
                      [  0.2767],
                      [ -0.8013],
                      [ -3.3813],
                      [ -6.0551],
                      [ -1.1033],
                      [ -4.2144],
                      [ -2.0643],
                      [ -1.2207],
                      [ -5.2666],
              